In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score

We want to combine the process

We like pipelines so that any new data goes through all the steps

In [ ]:
lr = LinearRegression()


ames = pd.read_csv("AmesHousing.csv")
X = ames[["Gr Liv Area", "TotRms AbvGrd"]]
y = ames["SalePrice"]



X_train, X_test, y_train, y_test = train_test_split(X, y)
#model expects standarized values

X_train_s = (X_train - X_train.mean())/X_train.std()

lr_fitted = lr.fit(X_train_s, y_train)
lr_fitted.coef_

array([ 73318.95740729, -19379.77355973])

WAtch as a mistake happens: no pipeline

In [ ]:
lr = LinearRegression()


ames = pd.read_csv("AmesHousing.csv")
X = ames[["Gr Liv Area", "TotRms AbvGrd"]]
y = ames["SalePrice"]



X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train_s = (X_train - X_train.mean())/X_train.std()

lr_fitted = lr.fit(X_train_s, y_train)
lr_fitted.coef_

array([ 67349.75731363, -15483.10484128])

In [ ]:
y_preds = lr_fitted.predict(X_test)

r2_score(y_test, y_preds)

-1687063.3810189117

R squares is negative 2 million--> soemthing really wrong with the model

In [ ]:
#predictions
y_preds[1:5]
# predictions are 100 million dollars

array([83331972.42197378, 60179139.01092756, 82487391.78657918,
       82508292.22936898])

We see that the data that was supposed to go into the eaquation was nto standardized

In [ ]:
#Another mistake
new_house = pd.DataFrame(data = {"Gr Liv Area": [889], "TotRms AbvGrd": [6]})
new_house
#standaridze values
new_house_s = (new_house - new_house.mean())/new_house.std()
new_house_s

,Gr Liv Area,TotRms AbvGrd
0,NaN,NaN


We get Nan bc we are tryig to get teh mean of teh same value, nothing else in the data frame

In [ ]:
X_test_s = (X_test - X_train.mean())/X_train.std()
y_preds = lr_fitted.predict(X_test_s)

r2_score(y_test, y_preds)

0.5824850544392977

In [ ]:
new_house_s = (new_house - X_train.mean())/X_train.std()
lr_fitted.predict(new_house_s)

array([103748.20823148])

want to create a pipeline that puts data through the same process every time

In [ ]:
lr= LinearRegression()

In [ ]:
lr_pipeline = Pipeline(
  [("stand", StandardScaler()),
  ("lr", LinearRegression())]
)

lr_pipeline

Pipeline(steps=[('stand', StandardScaler()), ('lr', LinearRegression())])

In [ ]:
lr_pipeline = Pipeline(
  [("standardize", StandardScaler()),
  ("linear_regression", LinearRegression())]
)

lr_pipeline

Pipeline(steps=[('standardize', StandardScaler()),
                ('linear_regression', LinearRegression())])

In [ ]:
lr_pipeline_fitted = lr_pipeline.fit(X_train, y_train)

y_preds = lr_pipeline_fitted.predict(X_test)
r2_score(y_test, y_preds)

0.5824850544392977

In [ ]:
lr_pipeline_fitted.predict(new_house)

array([103748.20823148])

In [ ]:
#create a pipeline using size and number of rooms
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(
  [  #name steps
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop"
)


lr_pipeline = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", LinearRegression())]
)

lr_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('dummify',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Bldg Type']),
                                                 ('standardize',
                                                  StandardScaler(),
                                                  ['Gr Liv Area',
                                                   'TotRms AbvGrd'])])),
                ('linear_regression', LinearRegression())])

In [ ]:
X = ames.drop("SalePrice", axis = 1)
y = ames["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

lr_fitted = lr_pipeline.fit(X_train, y_train)

In [ ]:
ct_fitted = ct.fit(X_train)

ct.transform(X_train)

array([[ 1.        ,  0.        ,  0.        , ...,  0.        ,
        -2.11921782, -2.20848383],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.55369892,  0.34387754],
       [ 0.        ,  0.        ,  0.        , ...,  1.        ,
        -1.73850564, -2.20848383],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.22787674,  0.34387754],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.13729498,  0.34387754],
       [ 0.        ,  0.        ,  0.        , ...,  1.        ,
        -0.82241697, -0.2942128 ]])

In [ ]:
ct.transform(X_test)

array([[ 1.        ,  0.        ,  0.        , ...,  0.        ,
        -0.42187603, -0.2942128 ],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
        -0.65188964, -0.2942128 ],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.21994523,  0.34387754],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
        -0.02331797,  0.98196789],
       [ 0.        ,  0.        ,  1.        , ...,  0.        ,
         0.53387016,  1.62005823],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.24175687,  0.98196789]])

In [ ]:
#lr_pipeline_fitted.coef_ does not work bc not attribute
# What we need to do instead is find the step of the pipeline where the model fitting happened, and get those coefficients:
lr_pipeline_fitted.named_steps['linear_regression'].coef_
#find thru the name we gaev it ^

array([ 67334.42790468, -15479.58074772])

In [ ]:
ct_dummies = ColumnTransformer(
  [("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])],
  remainder = "passthrough"
).set_output(transform = "pandas")

ct_inter = ColumnTransformer(
  [
    ("interaction", PolynomialFeatures(interaction_only = True), ["remainder__TotRms AbvGrd", "dummify__Bldg Type_1Fam"]),
  ],
  remainder = "drop"
).set_output(transform = "pandas")

X_train_dummified = ct_dummies.fit_transform(X_train)
X_train_dummified

,dummify__Bldg Type_1Fam,dummify__Bldg Type_2fmCon,dummify__Bldg Type_Duplex,dummify__Bldg Type_Twnhs,dummify__Bldg Type_TwnhsE,remainder__Order,remainder__PID,remainder__MS SubClass,remainder__MS Zoning,remainder__Lot Frontage,...,remainder__Screen Porch,remainder__Pool Area,remainder__Pool QC,remainder__Fence,remainder__Misc Feature,remainder__Misc Val,remainder__Mo Sold,remainder__Yr Sold,remainder__Sale Type,remainder__Sale Condition
907,1.0,0.0,0.0,0.0,0.0,908,909175080,30,RL,60.0,...,0,0,NaN,NaN,NaN,0,1,2009,WD,Normal
2127,1.0,0.0,0.0,0.0,0.0,2128,907180040,60,RL,69.0,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
977,0.0,0.0,0.0,0.0,1.0,978,923228110,180,RM,21.0,...,0,0,NaN,NaN,NaN,0,4,2009,WD,Normal
1668,1.0,0.0,0.0,0.0,0.0,1669,527404140,20,RL,70.0,...,0,0,NaN,GdWo,NaN,0,6,2007,WD,Normal
1431,1.0,0.0,0.0,0.0,0.0,1432,907126030,20,RL,65.0,...,0,0,NaN,NaN,NaN,0,8,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1157,1.0,0.0,0.0,0.0,0.0,1158,533125080,60,RL,NaN,...,196,0,NaN,NaN,NaN,0,6,2008,WD,Normal
1532,1.0,0.0,0.0,0.0,0.0,1533,909276170,70,RL,74.0,...,0,0,NaN,NaN,NaN,0,8,2008,WD,Normal
834,1.0,0.0,0.0,0.0,0.0,835,906475070,60,RL,NaN,...,225,0,NaN,NaN,NaN,0,6,2009,WD,Abnorml
612,1.0,0.0,0.0,0.0,0.0,613,534429150,80,RL,85.0,...,0,0,NaN,NaN,NaN,0,9,2009,WD,Normal


Consider four possible models for predicting house prices:

1;Using only the size and number of rooms.

2:Using size, number of rooms, and building type.
3:Using size and building type, and their interaction.
4:Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.
Set up a pipeline for each of these four models.

Then, get predictions on the test set for each of your pipelines, and compute the root mean squared error. Which model performed best?

Note: You should only use the function train_test_split() one time in your code; that is, we should be predicting on the same test set for all three models.

In [ ]:
X = ames.drop("SalePrice", axis = 1)
y = ames["SalePrice"]

In [ ]:
#model 1: size and number of rooms

ct = ColumnTransformer(
  [
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop"
)


lr_pipeline = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", LinearRegression())]
)

lr_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('standardize',
                                                  StandardScaler(),
                                                  ['Gr Liv Area',
                                                   'TotRms AbvGrd'])])),
                ('linear_regression', LinearRegression())])

In [ ]:
score= cross_val_score(lr_pipeline, X, y, cv = 5, scoring = 'r2')
score.mean()

0.504208752508862

In [ ]:
#model 2
ct = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop"
)

lr_pipeline2 = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", LinearRegression())]
)

lr_pipeline2

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('dummify',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Bldg Type']),
                                                 ('standardize',
                                                  StandardScaler(),
                                                  ['Gr Liv Area',
                                                   'TotRms AbvGrd'])])),
                ('linear_regression', LinearRegression())])

In [ ]:
score2= cross_val_score(lr_pipeline2, X, y, cv = 5, scoring = 'r2')
score2.mean()

0.533394662313337

In [ ]:
ct = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder= "passthrough"
).set_output(transform = "pandas")



In [ ]:
ct.fit_transform(X_train)

,dummify__Bldg Type_1Fam,dummify__Bldg Type_2fmCon,dummify__Bldg Type_Duplex,dummify__Bldg Type_Twnhs,dummify__Bldg Type_TwnhsE,standardize__Gr Liv Area,standardize__TotRms AbvGrd,remainder__Order,remainder__PID,remainder__MS SubClass,...,remainder__Screen Porch,remainder__Pool Area,remainder__Pool QC,remainder__Fence,remainder__Misc Feature,remainder__Misc Val,remainder__Mo Sold,remainder__Yr Sold,remainder__Sale Type,remainder__Sale Condition
907,1.0,0.0,0.0,0.0,0.0,-2.119218,-2.208484,908,909175080,30,...,0,0,NaN,NaN,NaN,0,1,2009,WD,Normal
2127,1.0,0.0,0.0,0.0,0.0,0.553699,0.343878,2128,907180040,60,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
977,0.0,0.0,0.0,0.0,1.0,-1.738506,-2.208484,978,923228110,180,...,0,0,NaN,NaN,NaN,0,4,2009,WD,Normal
1668,1.0,0.0,0.0,0.0,0.0,-1.258650,-0.932303,1669,527404140,20,...,0,0,NaN,GdWo,NaN,0,6,2007,WD,Normal
1431,1.0,0.0,0.0,0.0,0.0,-1.024670,-0.932303,1432,907126030,20,...,0,0,NaN,NaN,NaN,0,8,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1157,1.0,0.0,0.0,0.0,0.0,2.332339,1.620058,1158,533125080,60,...,196,0,NaN,NaN,NaN,0,6,2008,WD,Normal
1532,1.0,0.0,0.0,0.0,0.0,0.303857,-0.294213,1533,909276170,70,...,0,0,NaN,NaN,NaN,0,8,2008,WD,Normal
834,1.0,0.0,0.0,0.0,0.0,1.227877,0.343878,835,906475070,60,...,225,0,NaN,NaN,NaN,0,6,2009,WD,Abnorml
612,1.0,0.0,0.0,0.0,0.0,0.137295,0.343878,613,534429150,80,...,0,0,NaN,NaN,NaN,0,9,2009,WD,Normal


In [ ]:
#model 3:size, builing type and interaction

ct3 = ColumnTransformer(
    transformers=[
        ("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
        ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"]),
    ],
    remainder="passthrough"
)

ct_inter = ColumnTransformer(
    transformers=[
        ("interaction1", PolynomialFeatures(interaction_only=False), ["dummify__Bldg Type_1Fam", "standardize__Gr Liv Area"]),
        ("interaction2", PolynomialFeatures(interaction_only=False), ["dummify__Bldg Type_2fmCon", "standardize__Gr Liv Area"]),
        ("interaction3", PolynomialFeatures(interaction_only=False), ["dummify__Bldg Type_Twnhs", "standardize__Gr Liv Area"]),
        ("interaction4", PolynomialFeatures(interaction_only=False), ["dummify__Bldg Type_Duplex", "standardize__Gr Liv Area"]),
        ("interaction5", PolynomialFeatures(interaction_only=False), ["dummify__Bldg Type_TwnhsE", "standardize__Gr Liv Area"]),
    ],
    remainder="drop"
)


# Create the linear regression pipeline
lr_pipeline3 = Pipeline(
    steps=[
        ("preprocessing", ct3),
        ("interaction", ct_inter),
        ("linear_regression", LinearRegression())]
).set_output(transform="pandas")
lr_pipeline3

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('dummify',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Bldg Type']),
                                                 ('standardize',
                                                  StandardScaler(),
                                                  ['Gr Liv Area',
                                                   'TotRms AbvGrd'])])),
                ('interaction',
                 ColumnTransformer(transformers=[('interaction1',
                                                  PolynomialFeatures(),
                                                  ['dummify__Bldg Type_1Fam',
                                                   'standardize__Gr Liv Area'])...
                                                  ['dummify__Bldg Type_2fmCon',
                                                   'standardize__Gr Liv Area']),
                                                 ('interaction3',
                                                  PolynomialFeatures(),
                                                  ['dummify__Bldg Type_Twnhs',
                                                   'standardize__Gr Liv Area']),
                                                 ('interaction4',
                                                  PolynomialFeatures(),
                                                  ['dummify__Bldg Type_Duplex',
                                                   'standardize__Gr Liv Area']),
                                                 ('interaction5',
                                                  PolynomialFeatures(),
                                                  ['dummify__Bldg Type_TwnhsE',
                                                   'standardize__Gr Liv '
                                                   'Area'])])),
                ('linear_regression', LinearRegression())])

In [ ]:

# doesnt work
# Fit and transform the first ColumnTransformer
X_transformed = ct.fit_transform(X_train)

# Apply interaction only on the relevant transformed features
X_interaction = ct_inter.fit_transform(pd.DataFrame(X_transformed, columns=ct.get_feature_names_out()))

In [ ]:
score3= cross_val_score(lr_pipeline3, X, y, cv = 5, scoring = 'r2')
score3.mean()

0.5458306394857146

Three appears to be the best model based on the highest rSquared.

In [ ]:
#model 4: Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building typ

In [ ]:
X = ames.drop("SalePrice", axis = 1)
y = ames["SalePrice"]

In [ ]:
ct4 = ColumnTransformer(
    [
        ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"]),
        ("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
    ],
    remainder="drop"
).set_output(transform="pandas")

ct4.fit_transform(X)

,standardize__Gr Liv Area,standardize__TotRms AbvGrd,dummify__Bldg Type_1Fam,dummify__Bldg Type_2fmCon,dummify__Bldg Type_Duplex,dummify__Bldg Type_Twnhs,dummify__Bldg Type_TwnhsE
0,0.309265,0.354167,1.0,0.0,0.0,0.0,0.0
1,-1.194427,-0.917535,1.0,0.0,0.0,0.0,0.0
2,-0.337718,-0.281684,1.0,0.0,0.0,0.0,0.0
3,1.207523,0.990018,1.0,0.0,0.0,0.0,0.0
4,0.255844,-0.281684,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2925,-0.982723,-0.281684,1.0,0.0,0.0,0.0,0.0
2926,-1.182556,-0.917535,1.0,0.0,0.0,0.0,0.0
2927,-1.048015,-0.281684,1.0,0.0,0.0,0.0,0.0
2928,-0.219006,-0.281684,1.0,0.0,0.0,0.0,0.0


In [ ]:
#model 4
ct_poly = ColumnTransformer(
    [
        ("degree", PolynomialFeatures(degree=5), ["standardize__Gr Liv Area"]),
        ("degree2", PolynomialFeatures(degree=5), ["standardize__TotRms AbvGrd"]),
    ],
    remainder="drop"
)

#Create the linear regression pipeline
lr_pipeline4 = Pipeline(
    steps=[
        ("preprocessing", ct4),
        # ("polynomial", ct_poly),
        ("linear_regression", LinearRegression())
    ]
).set_output(transform="pandas")

lr_pipeline4

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('standardize',
                                                  StandardScaler(),
                                                  ['Gr Liv Area',
                                                   'TotRms AbvGrd']),
                                                 ('dummify',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Bldg Type'])])),
                ('linear_regression', LinearRegression())])

In [ ]:
score4= cross_val_score(lr_pipeline4, X, y, cv = 5, scoring = 'r2')
score4.mean()

0.5328824390692034

Consider one hundred modeling options for house price:

House size, trying degrees 1 through 10
Number of rooms, trying degrees 1 through 10
Building Type
Hint: The dictionary of possible values that you make to give to GridSearchCV will have two elements instead of one.

Q1: Which model performed the best?

Q2: What downsides do you see of trying all possible model options? How might you go about choosing a smaller number of tuning values to try?

In [ ]:
#for one variable
from sklearn.model_selection import GridSearchCV

ct_poly = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("poly_size", PolynomialFeatures(), ["Gr Liv Area"]),
    ("poly_room", PolynomialFeatures(), ["TotRms AbvGrd"]),
  ],
  remainder = "drop"
)

lr_pipeline_poly = Pipeline(
  [("preprocessing", ct_poly),
  ("linear_regression", LinearRegression())]
).set_output(transform="pandas")

degrees = {
    'preprocessing__poly_size__degree': np.arange(1, 10),
    'preprocessing__poly_room__degree': np.arange(1, 10)
}

gscv = GridSearchCV(lr_pipeline_poly, degrees, cv = 5, scoring='r2')
gscv_fitted = gscv.fit(X, y)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [ ]:
params_df = pd.DataFrame(gscv_fitted.cv_results_['params'])

results_df = params_df.assign(score=gscv.cv_results_['mean_test_score'])

results_df.sort_values(by='score', ascending=False)

,preprocessing__poly_room__degree,preprocessing__poly_size__degree,score
39,5,4,0.559335
30,4,4,0.559310
2,1,3,0.558259
11,2,3,0.557383
57,7,4,0.555899
...,...,...,...
62,7,9,-7.621862
53,6,9,-7.621862
44,5,9,-7.621862
35,4,9,-7.621862
